In [ ]:
  
# doVarMERA.py
# ---------------------------------------------------------------------
# Variational energy minimization of (scale-invariant) modified binary MERA
#
# by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 6/2019

import numpy as np
from numpy import linalg as LA
from ncon import ncon


def doVarMERA(hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC, chi, chimid, OPTS):
  """
  Variational energy minimization of (scale-invariant) modified binary MERA
  for nearest neighbour 1D Hamiltonian. Inputs 'hamAB, hamBA, rhoAB, rhoBA,
  wC, vC, uC' are lists whose lengths are equal to the number of MERA
  levels. Input Hamiltonian specified through 'hamAB[0]' and 'hamBA[0]'.
  Bond dimensions specified through 'chi' and 'chimid'.

  OPTS is a dict containing the optional arguments:
    numiter: int=1000, number of variatonal iterations
    refsym: bool=True, impose reflection symmetry
    numtrans: int=2, number of transitional layers
    dispon: bool=True, specify wether or not to display convergence data
    E0: float=0.0, specify exact ground energy (if known)
    sciter: int=4, iterations of power method to find rho
  """
  if 'numiter' not in OPTS:
    OPTS['numiter'] = 1000
  if 'numtrans' not in OPTS:
    OPTS['numtrans'] = 2
  if 'refsym' not in OPTS:
    OPTS['refsym'] = True
  if 'dispon' not in OPTS:
    OPTS['dispon'] = True
  if 'sciter' not in OPTS:
    OPTS['sciter'] = 4
  if 'E0' not in OPTS:
    OPTS['E0'] = 0

  # Add extra layers if required
  totLv = OPTS['numtrans'] + 1
  for k in range(totLv - len(wC)):
    wC.append(wC[-1])
    vC.append(vC[-1])
    uC.append(uC[-1])

  for k in range(1 + totLv - len(hamAB)):
    hamAB.append(hamAB[-1])
    hamBA.append(hamBA[-1])
    rhoAB.append(rhoAB[-1])
    rhoBA.append(rhoBA[-1])

  # Expand tensors to new dimensions if required
  chiZ = np.zeros(totLv + 1, dtype=int)
  chiZ[0] = hamAB[0].shape[0]
  chimidZ = np.zeros(totLv + 1, dtype=int)
  chimidZ[0] = hamAB[0].shape[0]
  for k in range(totLv):
    chiZ[k + 1] = min(chi, chiZ[k] * chimidZ[k])
    chimidZ[k + 1] = min(chimid, chiZ[k])
    wC[k] = TensorExpand(wC[k], [chiZ[k], chimidZ[k + 1], chiZ[k + 1]])
    vC[k] = TensorExpand(vC[k], [chiZ[k], chimidZ[k + 1], chiZ[k + 1]])
    uC[k] = TensorExpand(uC[k],
                         [chiZ[k], chiZ[k], chimidZ[k + 1], chimidZ[k + 1]])
    hamAB[k + 1] = TensorExpand(
        hamAB[k + 1], [chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]])
    hamBA[k + 1] = TensorExpand(
        hamBA[k + 1], [chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]])
    rhoAB[k + 1] = TensorExpand(
        rhoAB[k + 1], [chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]])
    rhoBA[k + 1] = TensorExpand(
        rhoBA[k + 1], [chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]])

  # Ensure Hamiltonian is negative defined
  hamABstart = hamAB[0]
  hamBAstart = hamBA[0]
  bias = max(LA.eigvalsh(hamAB[0].reshape(chiZ[0]**2, chiZ[0]**2)))
  hamAB[0] = hamAB[0] - bias * np.eye(chiZ[0]**2).reshape(
      chiZ[0], chiZ[0], chiZ[0], chiZ[0])
  hamBA[0] = hamBA[0] - bias * np.eye(chiZ[0]**2).reshape(
      chiZ[0], chiZ[0], chiZ[0], chiZ[0])

  Energy = 0
  for k in range(OPTS['numiter']):
    # Find scale-invariant density matrix (via power method)
    for g in range(OPTS['sciter']):
      rhoABtemp, rhoBAtemp = DescendSuper(rhoAB[totLv], rhoBA[totLv],
                                          wC[totLv - 1], vC[totLv - 1],
                                          uC[totLv - 1], OPTS['refsym'])
      rhoAB[totLv] = 0.5 * (rhoABtemp + np.conj(
          rhoABtemp.transpose(2, 3, 0, 1))) / ncon([rhoABtemp], [[1, 2, 1, 2]])
      rhoBA[totLv] = 0.5 * (rhoBAtemp + np.conj(
          rhoBAtemp.transpose(2, 3, 0, 1))) / ncon([rhoBAtemp], [[1, 2, 1, 2]])
      if OPTS['refsym']:
        rhoAB[totLv] = 0.5 * rhoAB[totLv] + 0.5 * rhoAB[totLv].transpose(
            1, 0, 3, 2)
        rhoBA[totLv] = 0.5 * rhoBA[totLv] + 0.5 * rhoBA[totLv].transpose(
            1, 0, 3, 2)

    # Descend density matrix through all layers
    for p in range(totLv - 1, -1, -1):
      rhoAB[p], rhoBA[p] = DescendSuper(rhoAB[p + 1], rhoBA[p + 1], wC[p],
                                        vC[p], uC[p], OPTS['refsym'])

    # Compute energy and display
    if OPTS['dispon']:
      if np.mod(k, 10) == 1:
        Energy = (ncon([rhoAB[0], hamAB[0]], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                  ncon([rhoBA[0], hamBA[0]],
                       [[1, 2, 3, 4], [1, 2, 3, 4]])) / 4 + bias / 2
        ExpectX = ncon([
            rhoAB[0].reshape(2, 2, 2, 2, 2, 2, 2, 2),
            np.array([[0, 1], [1, 0]])
        ], [[4, 1, 2, 3, 5, 1, 2, 3], [4, 5]])

        print('Iteration: %d of %d, Energy: %f, Err: %e, Mag: %e' %
              (k, OPTS['numiter'], Energy, Energy - OPTS['E0'], ExpectX))

    # Optimise over all layers
    for p in range(totLv):
      if k > 9:
        uEnv = DisEnv(hamAB[p], hamBA[p], rhoBA[p + 1], wC[p], vC[p], uC[p],
                      OPTS['refsym'])
        if OPTS['refsym']:
          uEnv = uEnv + uEnv.transpose(1, 0, 3, 2)

        uC[p] = TensorUpdateSVD(uEnv, 2)

      if k > 1:
        wEnv = IsoEnvW(hamAB[p], hamBA[p], rhoBA[p + 1], rhoAB[p + 1], wC[p],
                       vC[p], uC[p])
        wC[p] = TensorUpdateSVD(wEnv, 2)
        if OPTS['refsym']:
          vC[p] = wC[p]
        else:
          vEnv = IsoEnvV(hamAB[p], hamBA[p], rhoBA[p + 1], rhoAB[p + 1], wC[p],
                         vC[p], uC[p])
          vC[p] = TensorUpdateSVD(vEnv, 2)

      hamAB[p + 1], hamBA[p + 1] = AscendSuper(hamAB[p], hamBA[p], wC[p], vC[p],
                                               uC[p], OPTS['refsym'])

  hamAB[0] = hamABstart
  hamBA[0] = hamBAstart

  return Energy, hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC


def AscendSuper(hamAB, hamBA, w, v, u, refsym):
  """ apply the average ascending superoperator to the Hamiltonian """

  indList1 = [[6, 4, 1, 2], [1, 3, -3], [6, 7, -1], [2, 5, 3, 9], [4, 5, 7, 10],
              [8, 9, -4], [8, 10, -2]]
  indList2 = [[3, 4, 1, 2], [5, 6, -3], [5, 7, -1], [1, 2, 6, 9], [3, 4, 7, 10],
              [8, 9, -4], [8, 10, -2]]
  indList3 = [[5, 7, 2, 1], [8, 9, -3], [8, 10, -1], [4, 2, 9, 3],
              [4, 5, 10, 6], [1, 3, -4], [7, 6, -2]]
  indList4 = [[3, 6, 2, 5], [2, 1, -3], [3, 1, -1], [5, 4, -4], [6, 4, -2]]

  hamBAout = ncon(
      [hamAB, w, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList1)
  if refsym:
    hamBAout = hamBAout + hamBAout.transpose(1, 0, 3, 2)
  else:
    hamBAout = hamBAout + ncon(
        [hamAB, w, np.conj(w), u,
         np.conj(u), v, np.conj(v)], indList3)

  hamBAout = hamBAout + ncon(
      [hamBA, w, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList2)
  hamABout = ncon([hamBA, v, np.conj(v), w, np.conj(w)], indList4)

  return hamABout, hamBAout


def DescendSuper(rhoAB, rhoBA, w, v, u, refsym):
  """ apply the average descending superoperator to the density matrix """

  indList1 = [[9, 3, 4, 2], [-3, 5, 4], [-1, 10, 9], [-4, 7, 5, 6],
              [-2, 7, 10, 8], [1, 6, 2], [1, 8, 3]]
  indList2 = [[3, 6, 2, 5], [1, 7, 2], [1, 9, 3], [-3, -4, 7, 8],
              [-1, -2, 9, 10], [4, 8, 5], [4, 10, 6]]
  indList3 = [[3, 9, 2, 4], [1, 5, 2], [1, 8, 3], [7, -3, 5, 6], [7, -1, 8, 10],
              [-4, 6, 4], [-2, 10, 9]]
  indList4 = [[3, 6, 2, 5], [-3, 1, 2], [-1, 1, 3], [-4, 4, 5], [-2, 4, 6]]

  rhoABout = 0.5 * ncon(
      [rhoBA, w, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList1)
  if refsym:
    rhoABout = rhoABout + rhoABout.transpose(1, 0, 3, 2)
  else:
    rhoABout = rhoABout + 0.5 * ncon(
        [rhoBA, w, np.conj(w), u,
         np.conj(u), v, np.conj(v)], indList3)

  rhoBAout = 0.5 * ncon(
      [rhoBA, w, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList2)
  rhoBAout = rhoBAout + 0.5 * ncon(
      [rhoAB, v, np.conj(v), w, np.conj(w)], indList4)

  return rhoABout, rhoBAout


def DisEnv(hamAB, hamBA, rhoBA, w, v, u, refsym):
  """ compute the environment of a disentangler """

  indList1 = [[7, 8, 10, -1], [4, 3, 9, 2], [10, -3, 9], [7, 5, 4],
              [8, -2, 5, 6], [1, -4, 2], [1, 6, 3]]
  indList2 = [[7, 8, -1, -2], [3, 6, 2, 5], [1, -3, 2], [1, 9, 3],
              [7, 8, 9, 10], [4, -4, 5], [4, 10, 6]]
  indList3 = [[7, 8, -2, 10], [3, 4, 2, 9], [1, -3, 2], [1, 5, 3],
              [-1, 7, 5, 6], [10, -4, 9], [8, 6, 4]]

  uEnv = ncon(
      [hamAB, rhoBA, w, np.conj(w),
       np.conj(u), v, np.conj(v)], indList1)
  if refsym:
    uEnv = uEnv + uEnv.transpose(1, 0, 3, 2)
  else:
    uEnv = uEnv + ncon(
        [hamAB, rhoBA, w,
         np.conj(w), np.conj(u), v,
         np.conj(v)], indList3)

  uEnv = uEnv + ncon(
      [hamBA, rhoBA, w, np.conj(w),
       np.conj(u), v, np.conj(v)], indList2)

  return uEnv


def IsoEnvW(hamAB, hamBA, rhoBA, rhoAB, w, v, u):
  """ compute the environment of a 'w'-isometry """

  indList1 = [[7, 8, -1, 9], [4, 3, -3, 2], [7, 5, 4], [9, 10, -2, 11],
              [8, 10, 5, 6], [1, 11, 2], [1, 6, 3]]
  indList2 = [[1, 2, 3, 4], [10, 7, -3, 6], [-1, 11, 10], [3, 4, -2, 8],
              [1, 2, 11, 9], [5, 8, 6], [5, 9, 7]]
  indList3 = [[5, 7, 3, 1], [10, 9, -3, 8], [-1, 11, 10], [4, 3, -2, 2],
              [4, 5, 11, 6], [1, 2, 8], [7, 6, 9]]
  indList4 = [[3, 7, 2, -1], [5, 6, 4, -3], [2, 1, 4], [3, 1, 5], [7, -2, 6]]

  wEnv = ncon(
      [hamAB, rhoBA, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList1)
  wEnv = wEnv + ncon(
      [hamBA, rhoBA, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList2)
  wEnv = wEnv + ncon(
      [hamAB, rhoBA, np.conj(w), u,
       np.conj(u), v, np.conj(v)], indList3)
  wEnv = wEnv + ncon([hamBA, rhoAB, v, np.conj(v), np.conj(w)], indList4)

  return wEnv


def IsoEnvV(hamAB, hamBA, rhoBA, rhoAB, w, v, u):
  """ compute the environment of a 'v'-isometry """

  indList1 = [[6, 4, 1, 3], [9, 11, 8, -3], [1, 2, 8], [6, 7, 9], [3, 5, 2, -2],
              [4, 5, 7, 10], [-1, 10, 11]]
  indList2 = [[3, 4, 1, 2], [8, 10, 9, -3], [5, 6, 9], [5, 7, 8], [1, 2, 6, -2],
              [3, 4, 7, 11], [-1, 11, 10]]
  indList3 = [[9, 10, 11, -1], [3, 4, 2, -3], [1, 8, 2], [1, 5, 3],
              [7, 11, 8, -2], [7, 9, 5, 6], [10, 6, 4]]
  indList4 = [[7, 5, -1, 4], [6, 3, -3, 2], [7, -2, 6], [4, 1, 2], [5, 1, 3]]

  vEnv = ncon(
      [hamAB, rhoBA, w, np.conj(w), u,
       np.conj(u), np.conj(v)], indList1)
  vEnv = vEnv + ncon(
      [hamBA, rhoBA, w, np.conj(w), u,
       np.conj(u), np.conj(v)], indList2)
  vEnv = vEnv + ncon(
      [hamAB, rhoBA, w, np.conj(w), u,
       np.conj(u), np.conj(v)], indList3)
  vEnv = vEnv + ncon([hamBA, rhoAB, np.conj(v), w, np.conj(w)], indList4)

  return vEnv


def TensorExpand(A, chivec):
  """ expand tensor dimension by padding with zeros """

  if [*A.shape] == chivec:
    return A
  else:
    for k in range(len(chivec)):
      if A.shape[k] != chivec[k]:
        indloc = list(range(-1, -len(chivec) - 1, -1))
        indloc[k] = 1
        A = ncon([A, np.eye(A.shape[k], chivec[k])], [indloc, [1, -k - 1]])

    return A


def TensorUpdateSVD(wIn, leftnum):
  """ update an isometry using its (linearized) environment """

  wSh = wIn.shape
  ut, st, vht = LA.svd(
      wIn.reshape(np.prod(wSh[0:leftnum:1]), np.prod(wSh[leftnum:len(wSh):1])),
      full_matrices=False)
  return -(ut @ vht).reshape(wSh)



###################################################################################################################




  
# doConformalMERA.py
# ---------------------------------------------------------------------
# Extraction of conformal data from an optimised MERA
#
# by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 6/2019

import numpy as np
from numpy import linalg as LA
from scipy.sparse.linalg import eigs
from ncon import ncon


def doConformalMERA(wS, uS, vS, rhoBAS, scnum):
  """
  Compute conformal data from an modified binary MERA optimized for a
  scale-invariant critical point. Input 'ws', 'vs' and 'uS' are the
  isometries and disentangler from the scale-invariant layers, while
  'rhoBAS' is the fixed point density matrix. 'scnum' sets the number of
  scaling dimensions to compute.

  Outputs 'scDims', 'scOps', and 'Cfusion' are the scaling dimensions,
  scaling operators and fusion coefficients respectively.
  """

  # Diagonalize 1-site scaling superoperator
  chi = wS.shape[2]
  tensors = [wS, np.conj(wS), vS, np.conj(vS)]
  connects = [[-4, 1, 3], [-3, 1, 4], [3, 2, -2], [4, 2, -1]]
  ScSuper1 = ncon(tensors, connects).reshape(chi**2, chi**2)

  dtemp, utemp = eigs(ScSuper1, k=scnum, which='LM')
  scDims = -np.log2(abs(dtemp)) / 2

  # Normalize scaling operators
  scOps = [0 for x in range(scnum)]
  for k in range(scnum):
    scAtemp = utemp[:, k].reshape(chi, chi)
    scAtemp = scAtemp / LA.norm(scAtemp)

    tensors = [scAtemp, scAtemp, wS, np.conj(wS), uS, np.conj(uS), vS,
               np.conj(vS), rhoBAS]
    connects = [[8, 7], [3, 1], [7, 9, 11], [8, 10, 13], [2, 1, 9, 5],
                [2, 3, 10, 6], [4, 5, 12], [4, 6, 14], [13, 14, 11, 12]]
    cweight = ncon(tensors, connects)
    scOps[k] = scAtemp / np.sqrt(cweight)

  # Compute fusion coefficients (OPE coefficients)
  Cfusion = np.zeros((scnum, scnum, scnum), dtype=complex)
  for k1 in range(scnum):
    for k2 in range(scnum):
      for k3 in range(scnum):
        Otemp = scDims[k1] - scDims[k2] + scDims[k3]
        tensors = [scOps[k1], scOps[k2], scOps[k3], wS, np.conj(wS), uS,
                   np.conj(uS), vS, np.conj(vS), uS, np.conj(uS), wS,
                   np.conj(wS), wS, np.conj(wS), vS, np.conj(vS), rhoBAS]
        connects = [[5, 4], [3, 1], [28, 27], [4, 6, 11], [5, 7, 13],
                    [2, 1, 6, 9], [2, 3, 7, 10], [8, 9, 12], [8, 10, 14],
                    [11, 12, 16, 21], [13, 14, 17, 23], [15, 16, 18],
                    [15, 17, 19], [27, 26, 24], [28, 26, 25], [24, 21, 20],
                    [25, 23, 22], [19, 22, 18, 20]]
        Cfusion[k1, k2, k3] = (2**Otemp) * ncon(tensors, connects)

  return scDims, scOps, Cfusion



##################################################################################################################


  
# mainVarMERA.py
# ---------------------------------------------------------------------
# Script file for initializing the Hamiltonian and MERA tensors before
# passing to a variational energy minimization routine. Initiates the
# extraction of conformal data from MERA after the minimization is complete.
#
# by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 6/2019


# Preamble
import numpy as np
import matplotlib.pyplot as plt

from doVarMERA import doVarMERA
from doConformalMERA import doConformalMERA

# Example 1: crit Ising model #####
#######################################

# Set bond dimensions and simulation options
chi = 6
chimid = 4

OPTS = {'numiter': 2000,  # number of variatonal iterations
        'refsym': True,  # impose reflection symmetry
        'numtrans': 1,  # number of transitional layers
        'dispon': True,  # display convergence data
        'E0': -4 / np.pi,  # specify exact ground energy (if known)
        'sciter': 4}  # iterations of power method to find density matrix

# Define Hamiltonian (quantum critical Ising), do preliminary 2->1 blocking
OPTS['numtrans'] = int(max(OPTS['numtrans'],
                           np.ceil(np.log(chi) / (2 * np.log(4)))))
sX = np.array([[0, 1], [1, 0]], dtype=float)
sZ = np.array([[1, 0], [0, -1]], dtype=float)
htemp = -np.kron(sX, sX) - 0.5 * (
    np.kron(sZ, np.eye(2)) + np.kron(np.eye(2), sZ))
hbig = (0.5 * np.kron(np.eye(4), htemp) +
        np.kron(np.eye(2), np.kron(htemp, np.eye(2))) +
        0.5 * np.kron(htemp, np.eye(4))).reshape(2, 2, 2, 2, 2, 2, 2, 2)
hamAB = [0] * (OPTS['numtrans'] + 2)
hamBA = [0] * (OPTS['numtrans'] + 2)
hamAB[0] = (hbig.transpose(0, 1, 3, 2, 4, 5, 7, 6)).reshape(4, 4, 4, 4)
hamBA[0] = (hbig.transpose(1, 0, 2, 3, 5, 4, 6, 7)).reshape(4, 4, 4, 4)

# Initialize tensors
totLv = OPTS['numtrans'] + 1
chiZ = np.zeros(totLv + 1, dtype=int)
chiZ[0] = hamAB[0].shape[0]
chimidZ = np.zeros(totLv + 1, dtype=int)
chimidZ[0] = hamAB[0].shape[0]
for k in range(totLv):
  chiZ[k + 1] = min(chi, chiZ[k] * chimidZ[k])
  chimidZ[k + 1] = min(chimid, chiZ[k + 1])

wC = [0] * (OPTS['numtrans'] + 1)
vC = [0] * (OPTS['numtrans'] + 1)
uC = [0] * (OPTS['numtrans'] + 1)
for k in range(totLv):
  wC[k] = np.random.rand(chiZ[k], chimidZ[k], chiZ[k + 1])
  vC[k] = np.random.rand(chiZ[k], chimidZ[k], chiZ[k + 1])
  uC[k] = (np.eye(chiZ[k]**2, chimidZ[k]**2)).reshape(chiZ[k], chiZ[k],
                                                      chimidZ[k], chimidZ[k])

rhoAB = [0] * (OPTS['numtrans'] + 2)
rhoBA = [0] * (OPTS['numtrans'] + 2)
rhoAB[0] = np.eye(chiZ[0]**2).reshape(chiZ[0], chiZ[0], chiZ[0], chiZ[0])
rhoBA[0] = np.eye(chiZ[0]**2).reshape(chiZ[0], chiZ[0], chiZ[0], chiZ[0])
for k in range(totLv):
  rhoAB[k + 1] = np.eye(chiZ[k + 1]**2).reshape(chiZ[k + 1], chiZ[k + 1],
                                                chiZ[k + 1], chiZ[k + 1])
  rhoBA[k + 1] = np.eye(chiZ[k + 1]**2).reshape(chiZ[k + 1], chiZ[k + 1],
                                                chiZ[k + 1], chiZ[k + 1])
  hamAB[k + 1] = np.zeros((chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]))
  hamBA[k + 1] = np.zeros((chiZ[k + 1], chiZ[k + 1], chiZ[k + 1], chiZ[k + 1]))

# Perform variational optimization
Energy, hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC = doVarMERA(
    hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC, chi, chimid, OPTS)

# Expand bond dimension and increase number of transitional layers,
# then continue variational optimization
chi = 8
chimid = 6
OPTS['numtrans'] = 2
OPTS['numiter'] = 1800
Energy, hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC = doVarMERA(
    hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC, chi, chimid, OPTS)

# Expand bond dimension and increase number of transitional layers,
# then continue variational optimization
chi = 12
chimid = 8
OPTS['numtrans'] = 3
OPTS['numiter'] = 1400
Energy, hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC = doVarMERA(
    hamAB, hamBA, rhoAB, rhoBA, wC, vC, uC, chi, chimid, OPTS)

# Compute the conformal data from the optimized MERA
scnum = 10
scDims, scOps, Cfusion = doConformalMERA(wC[-1], uC[-1], vC[-1],
                                         rhoBA[-1], scnum)

# Compare with known results for Ising CFT
scDimsExact = [0, 1 / 8, 1, 1 + 1 / 8, 1 + 1 / 8, 2, 2, 2, 2, 2 + 1 / 8]
plt.figure(1)
plt.plot(range(scnum), scDimsExact, 'b', label="exact")
plt.plot(range(scnum), scDims, 'rx', label="MERA")
plt.legend()
plt.title('critical Ising model')
plt.xlabel('k')
plt.ylabel('Scaling Dims: Delta_k')
plt.show()

Cess = [Cfusion[1, 1, 2], Cfusion[1, 2, 1], Cfusion[2, 1, 1]]
CessExact = [0.5, 0.5, 0.5]
print('Fusion coefficients: Exact_C(ep,sg,sg) = %f, MERA_C(ep,sg,sg) = %f' %
      (CessExact[1], np.real(Cess[1])))

# Save data
np.save('IsingData.npy', (wC, uC, vC, rhoAB, rhoBA))





